In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import hddm
import numpy as np
import patsy 

samples = 30000

data = hddm.load_csv('../../../data/1_behav/DataAll_pupil.csv')

In [ ]:
def z_link_func(x, data=data):
    return 1 / (1 + np.exp(-(x)))

In [ ]:
### Patch to run PPC
import pymc as pm
import pymc.progressbar as pbar

def _parents_to_random_posterior_sample(bottom_node, pos=None):
    """Walks through parents and sets them to pos sample."""
    for i, parent in enumerate(bottom_node.extended_parents):
        if not isinstance(parent, pm.Node): # Skip non-stochastic nodes
            continue

        if pos is None:
            # Set to random posterior position
            pos = np.random.randint(0, len(parent.trace()))

        assert len(parent.trace()) >= pos, "pos larger than posterior sample size"
        parent.value = parent.trace()[pos]

def _post_pred_generate(bottom_node, samples=500, data=None, append_data=True):
    """Generate posterior predictive data from a single observed node."""
    datasets = []
    ##############################
    # Sample and generate stats
    for sample in range(samples):
        _parents_to_random_posterior_sample(bottom_node)
        # Generate data from bottom node
        sampled_data = bottom_node.random()
        if append_data and data is not None:
            sampled_data.reset_index(inplace=True)  # Only modification of original Kabuki code
            sampled_data = sampled_data.join(data.reset_index(), lsuffix='_sampled')
        datasets.append(sampled_data)
    return datasets

def post_pred_gen(model, groupby=None, samples=500, append_data=False, progress_bar=True):
    results = {}

    # Progress bar
    if progress_bar:
        n_iter = len(model.get_observeds())
        bar = pbar.progress_bar(n_iter)
        bar_iter = 0
    else:
        print("Sampling...")

    if groupby is None:
        iter_data = ((name, model.data.ix[obs['node'].value.index]) for name, obs in model.iter_observeds())
    else:
        iter_data = model.data.groupby(groupby)

    for name, data in iter_data:
        node = model.get_data_nodes(data.index)

        if progress_bar:
            bar_iter += 1
            bar.update(bar_iter)

        if node is None or not hasattr(node, 'random'):
            continue # Skip

        ##############################
        # Sample and generate stats
        datasets = _post_pred_generate(node, samples=samples, data=data, append_data=append_data)
        results[name] = pd.concat(datasets, names=['sample'], keys=list(range(len(datasets))))

    if progress_bar:
        bar_iter += 1
        bar.update(bar_iter)

    return pd.concat(results, names=['node'])

## zbp_vbp

In [ ]:
v_reg = {'model': 'v ~ 1 + stim + pupil:condition + baseline:condition', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1 + pupil:condition + baseline:condition', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

zbp_vbp = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)

zbp_vbp.sample(samples, burn=samples/10, thin=2)

In [ ]:
zbp_vbp.dic_info['deviance'] + zbp_vbp.dic_info['pD'] + zbp_vbp.dic_info['pD']

In [ ]:
# Save all parameters
a, t = zbp_vbp.nodes_db.node[['a', 't']]

zpc,zbc,z0 = zbp_vbp.nodes_db.node[['z_pupil:condition','z_baseline:condition','z_Intercept']] 

vpc,vbc,vs,v0 = zbp_vbp.nodes_db.node[['v_pupil:condition','v_baseline:condition','v_stim','v_Intercept']] 

allParms = a.trace()
allParms = np.column_stack([allParms,t.trace()])
allParms = np.column_stack([allParms,zpc.trace(),zbc.trace(),z0.trace()]);
allParms = np.column_stack([allParms,vpc.trace(),vbc.trace(),vs.trace(),v0.trace()])

np.savetxt("../../../data/3_ddm/modeloutputs/trace_processed/zbp_vbp.csv", allParms , delimiter=",")

In [ ]:
stats = zbp_vbp.gen_stats()
stats.to_csv('../../../data/3_ddm/modeloutputs/subject_parms/zbp_vbp.csv')

In [ ]:
ppc_data = post_pred_gen(zbp_vbp,samples=100,append_data=True)
np.savetxt("../../../data/3_ddm/modeloutputs/ppc/zbp_vbp_ppc.csv", ppc_data, delimiter=",")

## zbp_vbp_condition

In [ ]:
v_reg = {'model': 'v ~ 1 + stim + condition + pupil:condition + baseline:condition', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1 + condition + pupil:condition + baseline:condition', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

zbp_vbp_condition = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)

zbp_vbp_condition.sample(samples, burn=samples/10, thin=2)

In [ ]:
# Save all parameters
a,t = zbp_vbp_condition.nodes_db.node[['a', 't']]

zc,zpc,zbc,z0 = zbp_vbp_condition.nodes_db.node[['z_condition','z_pupil:condition','z_baseline:condition','z_Intercept']] 

vc,vpc,vbc,vs,v0 = zbp_vbp_condition.nodes_db.node[['v_condition','v_pupil:condition','v_baseline:condition','v_stim','v_Intercept']] 

allParms = a.trace()
allParms = np.column_stack([allParms,t.trace()])
allParms = np.column_stack([allParms,zc.trace(),zpc.trace(),zbc.trace(),z0.trace()]);
allParms = np.column_stack([allParms,vc.trace(),vpc.trace(),vbc.trace(),vs.trace(),v0.trace()])

np.savetxt("../../../data/3_ddm/modeloutputs/trace_processed/zbp_vbp_condition.csv", allParms , delimiter=",")

## zbp_vbp_intertrial

In [ ]:
v_reg = {'model': 'v ~ 1 + stim + pupil:condition + baseline:condition', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1 + pupil:condition + baseline:condition', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

zbp_vbp_intertrial = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       include=('sv', 'st', 'sz'),
                                       group_only_regressors=False)

zbp_vbp_condition.sample(samples, burn=samples/10, thin=2)

In [ ]:
# Save all parameters
a, t = zbp_vbp_intertrial.nodes_db.node[['a', 't']]

zpc,zbc,z0 = zbp_vbp_intertrial.nodes_db.node[['z_pupil:condition','z_baseline:condition','z_Intercept']] 

vpc,vbc,vs,v0 = zbp_vbp_intertrial.nodes_db.node[['v_pupil:condition','v_baseline:condition','v_stim','v_Intercept']] 

allParms = a.trace()
allParms = np.column_stack([allParms,t.trace()])
allParms = np.column_stack([allParms,zpc.trace(),zbc.trace(),z0.trace()]);
allParms = np.column_stack([allParms,vpc.trace(),vbc.trace(),vs.trace(),v0.trace()])

np.savetxt("../../../data/3_ddm/modeloutputs/trace_processed/zbp_vbp_intertrial.csv", allParms , delimiter=",")

## Other models

In [ ]:
# zbp_vp
v_reg = {'model': 'v ~ 1 + stim + pupil:condition', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1 + pupil:condition + baseline:condition', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

zbp_vp = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)

zbp_vp.sample(samples, burn=samples/10, thin=2)

zbp_vp.dic_info['deviance'] + zbp_vp.dic_info['pD'] + zbp_vp.dic_info['pD']


# zb_vbp
v_reg = {'model': 'v ~ 1 + stim + pupil:condition + baseline:condition', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1 + baseline:condition', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

zb_vbp = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)

zb_vbp.sample(samples, burn=samples/10, thin=2)

# zp_vbp
v_reg = {'model': 'v ~ 1 + stim + pupil:condition + baseline:condition', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1 + pupil:condition', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

zp_vbp = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)

zp_vbp.sample(samples, burn=samples/10, thin=2)


zp_vbp.dic_info['deviance'] + zp_vbp.dic_info['pD'] + zp_vb.dic_info['pD']


# zb_vp
v_reg = {'model': 'v ~ 1 + stim + pupil:condition', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1 + baseline:condition', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

zb_vp = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)

zb_vp.sample(samples, burn=samples/10, thin=2)

zb_vp.dic_info['deviance'] + zb_vp.dic_info['pD'] + zb_vp.dic_info['pD']

# zp_vp
v_reg = {'model': 'v ~ 1 + stim + pupil:condition', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1 + pupil:condition', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

zp_vp = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)

zp_vp.sample(samples, burn=samples/10, thin=2)

zp_vp.dic_info['deviance'] + zp_vp.dic_info['pD'] + zp_vp.dic_info['pD']

In [ ]:
# vbp
v_reg = {'model': 'v ~ 1 + stim + baseline:condition + pupil:condition', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

vbp = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)

vbp.sample(samples, burn=samples/10, thin=2)

vbp.dic_info['deviance'] + vbp.dic_info['pD'] + vbp.dic_info['pD']


# zb
v_reg = {'model': 'v ~ 1 + stim', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1 + baseline:condition', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

zb = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)

zb.sample(samples, burn=samples/10, thin=2)

zb.dic_info['deviance'] + zb.dic_info['pD'] + zb.dic_info['pD']


# zp
v_reg = {'model': 'v ~ 1 + stim', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1 + pupil:condition', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

zp = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)

zp.sample(samples, burn=samples/10, thin=2)

zp.dic_info['deviance'] + zp.dic_info['pD'] + zp.dic_info['pD']

# vb
v_reg = {'model': 'v ~ 1 + stim + baseline:condition', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

vb = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)

vb.sample(samples, burn=samples/10, thin=2)

vb.dic_info['deviance'] + vb.dic_info['pD'] + vb.dic_info['pD']

# vp
v_reg = {'model': 'v ~ 1 + stim + pupil:condition', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

vp = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)

vp.sample(samples, burn=samples/10, thin=2)

vp.dic_info['deviance'] + vp.dic_info['pD'] + vp.dic_info['pD']

# null
v_reg = {'model': 'v ~ 1 + stim', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

null_pupil = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)

null_pupil.sample(samples, burn=samples/10, thin=2)

null_pupil.dic_info['deviance'] + null_pupil.dic_info['pD'] + null_pupil.dic_info['pD']